# Code to measure the offset of each MIRI galaxy to the centre of the frame

To do this, we will read in the catalogue of galaxies to obtain their respective IDs, RAs and Decs. Then we will fit a centroid to all our MIRI images and determine their coordinates. From this we can calculate the spherical offsets. Lastly we will plot the results and see what happens.

The code is separated into several functions that are being called by one main function for better readability and easier debugging.

In [ ]:
%load_ext autoreload
%autoreload 2

import astropy.units as u
import numpy as np
import pandas as pd
import os
import glob
import warnings
import miri_utils.astrometry_utils as am


from astropy.table import Table
from matplotlib import pyplot as plt
from astropy.wcs import FITSFixedWarning

warnings.simplefilter("ignore", category=FITSFixedWarning)


Define the global catalogue:

In [ ]:
catalogue = '/home/bpc/University/master/Red_Cardinal/cat_targets.fits'

global_cat = Table.read(catalogue)

# Add the columns to store the astrometric offsets
for col in ["primer003_dra", "primer003_ddec", 
            "primer004_dra", "primer004_ddec", 
            "cweb1_dra", "cweb1_ddec",
            "cweb2_dra", "cweb2_ddec"]:
    if col not in global_cat.colnames:
        global_cat[col] = 0.0 * u.arcsec

global_cat

Load the necessary data files

In [ ]:

# Specify the path to the cutouts directory
cutouts = '/home/bpc/University/master/Red_Cardinal/cutouts/'
coated = '/home/bpc/University/master/Red_Cardinal/cutoutsated/'


In [ ]:
#compute_offset(co_rotated, 'primer', 'F770W', '003')
#compute_offset(co_rotated, 'primer', 'F770W', '004')
#compute_offset(co_rotated, 'primer', 'F1800W', '003')
#compute_offset(co_rotated, 'primer', 'F1800W', '004')
#compute_offset(co_rotated, survey='cweb', filter='F770W', obs="1")
#compute_offset(co_rotated, survey='cweb', filter='F770W', obs="2")

out_dir = 'offsets2'
am.compute_offset(cutouts, out_dir, 'primer', 'F770W', '003')
am.compute_offset(cutouts, out_dir, 'primer', 'F770W', '004')
#compute_offset(cutouts, out_dir, 'primer', 'F1800W', '003')
#compute_offset(cutouts, out_dir, 'primer', 'F1800W', '004')
am.compute_offset(cutouts, out_dir, survey='cweb', filter='F770W', obs="1")
am.compute_offset(cutouts, out_dir, survey='cweb', filter='F770W', obs="2")

Now we visualise the catalogue and store it in a csv-file

In [ ]:
global_cat
fname = "/home/bpc/University/master/Red_Cardinal/offsets2/astrometric_offsets_orig.csv"
global_cat.write(fname, format="csv", overwrite=True)

# Function to visualise the offsets

Call the function!

In [ ]:
# Read in the data as a pandas dataframe
fname = "/home/bpc/University/master/Red_Cardinal/offsets2/astrometric_offsets_orig.csv"
df = pd.read_csv(fname)

# Exclude the following galaxies from analysis
exclude_ids = [19098, 19681, 21451, 7934, 8465, 9517, 10415, 11247, 11451, 12133,
               12175, 12213, 7696, 9809, 10600, 11137, 16615, 16874, 17517, 11481,
               12443, 20720, 21472, 21547, 22606]

exclude_ids_orig = [18769, 19098, 19681, 7904, 8465, 9517, 10415, 11247, 12133, 
                    12175, 7696, 16615, 17517, 17534, 17669, 19307, 8843, 9809, 
                    9986, 11137, 11481, 12133, 12175, 12443, 12513, 20720, 21472,
                    21547, 22606]
#exclude_ids_F1800W = [17793, 18769, 19098, 19681, 21451]

# Specify output directory
output_dir = "/home/bpc/University/master/Red_Cardinal/offsets2/"


# Call the function and store new DataFrames
df_primer003 = am.visualise_offsets(df, 'primer003', output_dir, exclude_ids_orig, 'F770W', 0.0)
df_primer004 = am.visualise_offsets(df, 'primer004', output_dir, exclude_ids_orig, 'F770W', 0.0)
#df_primer003_1800 = visualise_offsets(df, 'primer003', output_dir, exclude_ids_F1800W, 'F1800W', 0.0)
#df_primer004_1800 = visualise_offsets(df, 'primer004', output_dir, exclude_ids, 'F1800W', 0.0)
df_cweb1 = am.visualise_offsets(df, 'cweb1', output_dir, exclude_ids_orig, 'F770W', 0.0)
df_cweb2 = am.visualise_offsets(df, 'cweb2', output_dir, exclude_ids_orig, 'F770W', 0.0)


Produce scatter plots with all surveys included

In [ ]:
# Specify output directory
output_dir = "/home/bpc/University/master/Red_Cardinal/offsets2/plots/"



figname = output_dir + 'offset_scatter_orig.png'

plt.scatter(df_primer003['dra'], df_primer003['ddec'], label='PRIMER 003')
plt.scatter(df_primer004['dra'], df_primer004['ddec'], label='PRIMER 004')
plt.scatter(df_cweb1['dra'], df_cweb1['ddec'], label='COSMOS-Web 1')
plt.scatter(df_cweb2['dra'], df_cweb2['ddec'], label='COSMOS-Web 2')
plt.xlabel('Delta RA (arcsec)')
plt.ylabel('Delta dec (arcsec)')
plt.title('Astrometric offset from F444W cutout to F770W')
plt.xlim(-1, 1)
plt.ylim(-1, 1)
plt.legend()
plt.axvline(0, ls='--', color='k')
plt.axhline(0, ls='--', color='k')
plt.savefig(figname)
plt.show()


figname = output_dir + 'avg_offset_scatter_orig.png'

plt.scatter(np.mean(df_primer003['dra']), np.mean(df_primer003['ddec']), label='PRIMER 003')
plt.scatter(np.mean(df_primer004['dra']), np.mean(df_primer004['ddec']), label='PRIMER 004')
plt.scatter(np.mean(df_cweb1['dra']), np.mean(df_cweb1['ddec']), label='COSMOS-Web 1')
plt.scatter(np.mean(df_cweb2['dra']), np.mean(df_cweb2['ddec']), label='COSMOS-Web 2')
plt.xlabel('Delta RA (arcsec)')
plt.ylabel('Delta dec (arcsec)')
plt.title('Average astrometric offset from F444W cutout to F770W')
plt.xlim(-1, 1)
plt.ylim(-1, 1)
plt.legend()
plt.axvline(0, ls='--', color='k')
plt.axhline(0, ls='--', color='k')
plt.savefig(figname)
plt.show()



"""
figname = output_dir + 'offset_F1800W_scatter.png'

plt.scatter(df_primer003_1800['dra'], df_primer003_1800['ddec'], label='PRIMER 003')

plt.xlabel('Delta RA (arcsec)')
plt.ylabel('Delta dec (arcsec)')
plt.title('Astrometric offset from F444W cutout to F1800W')
plt.xlim(-1, 1)
plt.ylim(-1, 1)
plt.legend()
plt.axvline(0, ls='--', color='k')
plt.axhline(0, ls='--', color='k')
plt.savefig(figname)
plt.show()


figname = output_dir + 'avg_offset_F1800W_scatter.png'

plt.scatter(np.mean(df_primer003_1800['dra']), np.mean(df_primer003_1800['ddec']), label='PRIMER 003')
plt.xlabel('Delta RA (arcsec)')
plt.ylabel('Delta dec (arcsec)')
plt.title('Average astrometric offset from F444W cutout to F1800W')
plt.xlim(-1, 1)
plt.ylim(-1, 1)
plt.legend()
plt.axvline(0, ls='--', color='k')
plt.axhline(0, ls='--', color='k')
plt.savefig(figname)
plt.show()


"""

# Define a function that compensates for the offsets

Now we shift the original FITS files such that they overlap with NIRCam, but are not rotated

In [ ]:

# Define input path
miri = "/home/bpc/University/master/Red_Cardinal/MIRI/"

# Define the offsets directory where the statistics files are
offset_dir = "/home/bpc/University/master/Red_Cardinal/offsets2/"

# Shift primer observation 003
primer003 = glob.glob(os.path.join(miri, "PRIMER_003/*.fits"))
print(f"Found {len(primer003)} FITS files for the PRIMER survey.")

for fits_file in primer003:
    stat_file = os.path.join(offset_dir, 'offset_orig_primer003_stats.json')
    dra, ddec = am.get_mean_stats(stat_file)
    am.shift_miri_fits(fits_file, dra, ddec)

# Shift PRIMER observation 004
primer004 = glob.glob(os.path.join(miri, "PRIMER_004/*.fits"))
print(f"Found {len(primer004)} FITS files for the PRIMER survey.")

for fits_file in primer004:
    stat_file = os.path.join(offset_dir, 'offset_orig_primer004_stats.json')
    dra, ddec = am.get_mean_stats(stat_file)
    am.shift_miri_fits(fits_file, dra, ddec)


# Shift the first COSMOS-Web tiles
cweb1 = glob.glob(os.path.join(miri, "COSMOS-Web_1/*.fits"))
print(f"Found {len(cweb1)} FITS files for the COSMOS-Web survey #1.")

for fits_file in cweb1:
    stat_file = os.path.join(offset_dir, 'offset_orig_cweb1_stats.json')
    dra, ddec = am.get_mean_stats(stat_file)
    am.shift_miri_fits(fits_file, dra, ddec)


# Shift the second COSMOS-Web tiles
cweb2 = glob.glob(os.path.join(miri, "COSMOS-Web_2/*.fits"))
print(f"Found {len(cweb2)} FITS files for the COSMOS-Web survey #2.")

for fits_file in cweb2:
    stat_file = os.path.join(offset_dir, 'offset_orig_cweb2_stats.json')
    dra, ddec = am.get_mean_stats(stat_file)
    am.shift_miri_fits(fits_file, dra, ddec)
